<a href="https://colab.research.google.com/github/FarzanaR11/AI-Horde/blob/main/YOLOv8_OBJECT_MONITORING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

YOLOv8 OBJECT MONITORING

https://www.youtube.com/watch?v=aBVGKoNZQUw&list=PLZCA39VpuaZZ1cjH4vEIdXIb0dCpZs3Y5&index=11
https://chatgpt.com/c/672ee5d1-4810-8013-9eb6-ca7c37b5917a

In [1]:
pip install ultralytics opencv-python numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 11.8 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
import time
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")  # Use 'yolov8n.pt' or a larger model if preferred


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 76.0MB/s]


In [4]:
import cv2
import numpy as np
import time
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")  # Use 'yolov8n.pt' or a larger model if preferred

# Define the zones as polygons using coordinates
# Replace with actual coordinates for your zones:
x1, y1 = 100, 100  # Example coordinates for Cabin 1
x2, y2 = 200, 100
x3, y3 = 200, 200
x4, y4 = 100, 200

x5, y5 = 300, 100  # Example coordinates for Cabin 2
x6, y6 = 400, 100
x7, y7 = 400, 200
x8, y8 = 300, 200

zones = [
    np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]]),  # Cabin 1
    np.array([[x5, y5], [x6, y6], [x7, y7], [x8, y8]]),  # Cabin 2
    # Add more zones as needed
]
zone_times = {i: 0 for i in range(len(zones))}
zone_entries = {i: {} for i in range(len(zones))}

In [5]:
# Start video capture
cap = cv2.VideoCapture(0)  # Use 0 for webcam or provide video file path

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 inference
    results = model(frame)

    # Get detections
    for result in results:
        for box in result.boxes:
            if box.cls == 0:  # '0' is usually the class ID for 'person'
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                person_center = ((x1 + x2) // 2, (y1 + y2) // 2)

                # Check which zone the person is in
                for zone_index, zone in enumerate(zones):
                    if cv2.pointPolygonTest(zone, person_center, False) >= 0:
                        if zone_index not in zone_entries:
                            zone_entries[zone_index][box.id] = time.time()
                        else:
                            entry_time = zone_entries[zone_index].get(box.id, time.time())
                            zone_times[zone_index] += time.time() - entry_time
                            zone_entries[zone_index][box.id] = time.time()

                        # Draw the zone and bounding box
                        cv2.polylines(frame, [zone], True, (0, 255, 0), 2)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                        cv2.putText(frame, f"Cabin {zone_index+1}: {int(zone_times[zone_index])}s",
                                    (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Show the frame
    cv2.imshow("Productivity Monitoring", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [6]:
import cv2
import numpy as np
import time
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")  # Use a larger model like 'yolov8s.pt' for higher accuracy if needed

# Define zones as polygons using coordinates (replace with your workspace zones)
zones = [
    np.array([[100, 100], [200, 100], [200, 200], [100, 200]]),  # Example coordinates for Zone 1
    np.array([[300, 100], [400, 100], [400, 200], [300, 200]]),  # Example coordinates for Zone 2
    # Add more zones as needed
]

# Initialize dictionaries to track time and entries
zone_times = {i: 0 for i in range(len(zones))}
zone_entries = {i: {} for i in range(len(zones))}

# Start video capture
cap = cv2.VideoCapture(0)  # Use 0 for webcam or provide a video file path

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 inference
    results = model(frame)

    # Get current time for time tracking
    current_time = time.time()

    # Iterate over each detection
    for result in results:
        for box in result.boxes:
            if box.cls == 0:  # '0' is usually the class ID for 'person' in YOLO
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                person_center = ((x1 + x2) // 2, (y1 + y2) // 2)

                # Check if the person is in any zone
                for zone_index, zone in enumerate(zones):
                    if cv2.pointPolygonTest(zone, person_center, False) >= 0:
                        # If person is in the zone, update time
                        if box.id not in zone_entries[zone_index]:
                            zone_entries[zone_index][box.id] = current_time
                        else:
                            entry_time = zone_entries[zone_index][box.id]
                            zone_times[zone_index] += current_time - entry_time
                            zone_entries[zone_index][box.id] = current_time

                        # Draw the zone and bounding box
                        cv2.polylines(frame, [zone], True, (0, 255, 0), 2)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                        cv2.putText(
                            frame,
                            f"Cabin {zone_index + 1}: {int(zone_times[zone_index])}s",
                            (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            (255, 255, 255),
                            2,
                        )

    # Display the video feed with annotations
    cv2.imshow("Productivity Monitoring", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
